In [1]:
import numpy as np
from ase import Atoms
from ase.build import bulk
from ase.visualize import view
from dftpy.grid import DirectGrid
from dftpy.field import DirectField
from dftpy.functional import Functional, TotalFunctional, FunctionalOutput
from dftpy.functional.abstract_functional import AbstractFunctional
from dftpy.ions import Ions
from dftpy.constants import Units
from dftpy.formats import io
from dftpy.mpi import mp
from ase.io import read
from dftpy.math_utils import ecut2nr
from dftpy.optimization import Optimization


path_pp='/Users/valeria/Documents/PP/ofpp/EAC/upf/blps/'
file='si.lda.upf'
PP_list = {'Si': path_pp+file}

In [2]:
import os
os.chdir('../..')
path_file = os.getcwd()

In [3]:
dictionary = {
    'Si_Btin': {'file': 'Si_Btin.vasp'}, 
    'Si_fcc': {'file': 'Si_fcc.vasp'},
    'Si_bcc': {'file': 'Si_bcc.vasp'},
    'Si_cd': {'file': 'Si_cd.vasp'},
    'Si_dhcp': {'file': 'Si_dhcp.vasp'},
    'Si_bct5': {'file': 'Si_bct5.vasp'},
    'Si_sh': {'file': 'Si_sh.vasp'},
    'Si_cbcc': {'file': 'Si_cbcc.vasp'},
    'Si_hcp': {'file': 'Si_hcp.vasp'},
}

def get_rho_ini(phase, charge):
    inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
    ions = Ions.from_ase(read(inputfile, format='vasp'))
    fu_ks = io.read_density(path_file+'/Results/Fukui/'+str(phase)+'_fukui.xsf')
    nr = fu_ks.grid.nr
#     print(phase, nr)
    grid = DirectGrid(lattice=ions.cell, nr=nr)
    if charge is None:
        pseudo = Functional(type='PSEUDO', grid = grid, ions=ions, PP_list=PP_list)
        rho_ini = DirectField(grid=grid)
        rho_ini[:] = ions.get_ncharges()/ions.cell.volume
    else:
        pseudo = Functional(type='PSEUDO', grid = grid, ions=ions, PP_list=PP_list)        
        rho_ini = DirectField(grid=grid)
        rho_ini[:] = (ions.get_ncharges()-charge)/ions.cell.volume
    return rho_ini, grid, pseudo, ions

def get_rho(pseudo, rho_ini, rho0):
    ke = Functional(type='KEDF',name='WT', rho0= rho0)
    evaluator = TotalFunctional(KE=ke, XC=xc, HARTREE=hartree, PSEUDO=pseudo)
    optimization_options = {'econv' : 1e-7}
    opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
            optimization_method = 'TN')
    Rho_opt = opt.optimize_rho(guess_rho=rho_ini)
    return Rho_opt

def fukui_fun(rho_n, rho_dn):
    return (rho_dn - rho_n)/0.1

In [4]:
Phases = ['Si_Btin', 'Si_fcc', 'Si_bcc', 'Si_cd', 'Si_dhcp', 'Si_hcp', 'Si_bct5', 'Si_sh']   
# Phases = ['Si_bcc']
Rho_ini = []
Grid = []
Pseudo = []
IONS = []

Rho_ini_charged = []
Grid_charged = []
Pseudo_charged = []
IONS_charged = [] 
for i,phase in enumerate(Phases):
    rho_ini, grid, pseudo, ions = get_rho_ini(phase, charge=None)
    Rho_ini.append(rho_ini)
    Grid.append(grid)
    Pseudo.append(pseudo)
    IONS.append(ions)

    rho_ini, grid, pseudo, ions = get_rho_ini(phase, charge=0.1)
    Rho_ini_charged.append(rho_ini)
    Grid_charged.append(grid)
    Pseudo_charged.append(pseudo)
    IONS_charged.append(ions)    

setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key: Si -> /Users/valeria/Documents/PP/ofpp/EAC/upf/blps/si.lda.upf
setting key:

In [5]:
import h5py
file = h5py.File(path_file+"/organize_data/Data_phases.hdf5", "r")
fileX = file["Descriptor"]
rho0_KE_WT = file["rho0_KE_wt"]
rho0_TE_WT = file["rho0_TE_wt"]
rho0_DEN_WT = file["rho0_DEN_wt"]

rho0_KE_wt =[]
rho0_TE_wt =[]
rho0_DEN_wt=[]
rho = []
for i in np.arange(0,280):
    rho0_KE_wt.append(rho0_KE_WT[i][0]) 
    rho0_TE_wt.append(rho0_TE_WT[i][0]) 
    rho0_DEN_wt.append(rho0_DEN_WT[i][0])
file.close()
rho0_Btin_KE_wt = rho0_KE_wt[0:30];rho0_fcc_KE_wt = rho0_KE_wt[30:60];rho0_bcc_KE_wt = rho0_KE_wt[60:90];rho0_cd_KE_wt = rho0_KE_wt[90:120];rho0_dhcp_KE_wt = rho0_KE_wt[120:150];rho0_Btin_hd_KE_wt = rho0_KE_wt[150:180];rho0_rand4_KE_wt = rho0_KE_wt[180:185];rho0_rand8_KE_wt = rho0_KE_wt[185:190];rho0_bct5_KE_wt = rho0_KE_wt[190:220];rho0_hcp_KE_wt = rho0_KE_wt[220:250];rho0_sh_KE_wt = rho0_KE_wt[250:280]

rho0_Btin_TE_wt = rho0_TE_wt[0:30];rho0_fcc_TE_wt = rho0_TE_wt[30:60];rho0_bcc_TE_wt = rho0_TE_wt[60:90];rho0_cd_TE_wt = rho0_TE_wt[90:120];rho0_dhcp_TE_wt = rho0_TE_wt[120:150];rho0_Btin_hd_TE_wt = rho0_TE_wt[150:180];rho0_rand4_TE_wt = rho0_TE_wt[180:185];rho0_rand8_TE_wt = rho0_TE_wt[185:190];rho0_bct5_TE_wt = rho0_TE_wt[190:220];rho0_hcp_TE_wt = rho0_TE_wt[220:250];rho0_sh_TE_wt = rho0_TE_wt[250:280]

rho0_Btin_DEN_wt = rho0_DEN_wt[0:30];rho0_fcc_DEN_wt = rho0_DEN_wt[30:60];rho0_bcc_DEN_wt = rho0_DEN_wt[60:90];rho0_cd_DEN_wt = rho0_DEN_wt[90:120];rho0_dhcp_DEN_wt = rho0_DEN_wt[120:150];rho0_Btin_hd_DEN_wt = rho0_DEN_wt[150:180];rho0_rand4_DEN_wt = rho0_DEN_wt[180:185];rho0_rand8_DEN_wt = rho0_DEN_wt[185:190];rho0_bct5_DEN_wt = rho0_DEN_wt[190:220];rho0_hcp_DEN_wt = rho0_DEN_wt[220:250];rho0_sh_DEN_wt = rho0_DEN_wt[250:280]

In [6]:
rho0_TE = [rho0_Btin_TE_wt, rho0_fcc_TE_wt, rho0_bcc_TE_wt, rho0_cd_TE_wt, rho0_dhcp_TE_wt, rho0_hcp_TE_wt, rho0_bct5_TE_wt, rho0_sh_TE_wt]
rho0_KE = [rho0_Btin_KE_wt, rho0_fcc_KE_wt, rho0_bcc_KE_wt, rho0_cd_KE_wt, rho0_dhcp_KE_wt, rho0_hcp_KE_wt, rho0_bct5_KE_wt, rho0_sh_KE_wt]
rho0_DEN = [rho0_Btin_DEN_wt, rho0_fcc_DEN_wt, rho0_bcc_DEN_wt, rho0_cd_DEN_wt, rho0_dhcp_DEN_wt, rho0_hcp_DEN_wt, rho0_bct5_DEN_wt, rho0_sh_DEN_wt]

In [35]:
xc = Functional(type='XC',name='LDA')
hartree = Functional(type='HARTREE')
rho_opt_KE = []
rho_opt_TE = []
rho_opt_DEN = []
rho_opt_AVE = []
rho_opt_charged_KE = []
rho_opt_charged_TE = []
rho_opt_charged_DEN = []
rho_opt_charged_AVE = []
for i,phase in enumerate(Phases):
    rho_opt_KE.append(get_rho(Pseudo[i], Rho_ini[i], rho0_KE[i][10]))
    rho_opt_TE.append(get_rho(Pseudo[i], Rho_ini[i], rho0_TE[i][10]))
    rho_opt_DEN.append(get_rho(Pseudo[i], Rho_ini[i], rho0_DEN[i][10]))
    rho_opt_AVE.append(get_rho(Pseudo[i], Rho_ini[i], Rho_ini[i].mean()))
    
    rho_opt_charged_KE.append(get_rho(Pseudo[i], Rho_ini_charged[i], rho0_KE[i][10]))
    rho_opt_charged_TE.append(get_rho(Pseudo[i], Rho_ini_charged[i], rho0_TE[i][10]))
    rho_opt_charged_DEN.append(get_rho(Pseudo[i], Rho_ini_charged[i], rho0_DEN[i][10]))
    rho_opt_charged_AVE.append(get_rho(Pseudo[i], Rho_ini_charged[i], Rho_ini[i].mean()))

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.516417282922E+01     -1.516417E+01   2.736092E+00    1       1       1.991010E-02    
1       -1.606951955887E+01     -9.053467E-01   5.698768E-01    12      2       1.447051E-01    
2       -1.608254596650E+01     -1.302641E-02   3.566086E-02    8       1       2.233891E-01    
3       -1.608428912712E+01     -1.743161E-03   1.594507E-03    12      1       3.443811E-01    
4       -1.608438303555E+01     -9.390843E-05   1.159805E-04    15      1       4.755609E-01    
5       -1.608438860115E+01     -5.565595E-06   1.080577E-05    12      1       5.842049E-01    
6       -1.608438947947E+01     -8.783171E-07   1.019509E-06    16      1       7.090700E-01    
7       -1.608438951403E+01     -3.456567E-08   9.672961E-08    9       1       8.852789E-01    
8       -1.608438951927E+01     -5.237005E-09   8.534265E-09    15      1       1.021464E+00    
#### Density Optimization Conv

2       -1.610681375793E+01     -1.225321E-02   3.303830E-02    8       1       1.799431E-01    
3       -1.610842569716E+01     -1.611939E-03   1.437634E-03    12      1       2.865031E-01    
4       -1.610850713061E+01     -8.143345E-05   9.413436E-05    15      1       4.244611E-01    
5       -1.610851170099E+01     -4.570383E-06   8.866100E-06    12      1       5.228751E-01    
6       -1.610851239914E+01     -6.981485E-07   7.892625E-07    16      1       6.728530E-01    
7       -1.610851242538E+01     -2.624007E-08   7.330030E-08    9       1       7.682950E-01    
8       -1.610851242936E+01     -3.975792E-09   6.323670E-09    15      1       8.881991E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.34215303859537877
Chemical potential (eV)  : 9.310458412629076
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.526117300627E+01     -1.526117E+01   3.674299E+00    1       1       1.325321E-0

7       -1.601284688933E+01     -4.468736E-08   6.226317E-08    8       1       3.933556E-01    
8       -1.601284689381E+01     -4.480931E-09   4.828389E-09    13      1       4.638338E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.6136452175043035
Chemical potential (eV)  : 16.6981368955164
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.532720945234E+01     -1.532721E+01   3.651334E+00    1       1       6.477118E-03    
1       -1.596838936256E+01     -6.411799E-01   3.176305E-01    2       1       2.432108E-02    
2       -1.601187477534E+01     -4.348541E-02   2.435567E-02    15      1       1.082830E-01    
3       -1.601286242671E+01     -9.876514E-04   2.167637E-03    10      1       1.662719E-01    
4       -1.601298227144E+01     -1.198447E-04   1.982301E-04    13      1       2.388229E-01    
5       -1.601298939121E+01     -7.119778E-06   1.807119E-05    9       1       2.993391E-01 

4       -8.067577135177E+00     -9.444843E-04   1.314299E-03    14      1       2.278099E-01    
5       -8.067602435948E+00     -2.530077E-05   1.188877E-04    5       1       2.537231E-01    
6       -8.067607875069E+00     -5.439121E-06   1.206951E-05    13      1       3.107319E-01    
7       -8.067608780429E+00     -9.053600E-07   3.858495E-06    12      1       3.639860E-01    
8       -8.067608991216E+00     -2.107879E-07   7.581899E-07    12      1       4.168580E-01    
9       -8.067609059659E+00     -6.844203E-08   4.654492E-08    15      1       4.817190E-01    
10      -8.067609060759E+00     -1.100888E-09   2.783251E-09    6       1       5.106180E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.371178381549542
Chemical potential (eV)  : 10.10027822424446
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -7.738816498852E+00     -7.738816E+00   1.280679E+00    1       1       5.520105E-03 

8       -3.200094036415E+01     -4.988743E-08   2.515327E-07    12      1       2.232523E+00    
9       -3.200094036914E+01     -4.987449E-09   1.801439E-08    10      1       2.459844E+00    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.20451780806566466
Chemical potential (eV)  : 5.56521302413225
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -2.774513060974E+01     -2.774513E+01   7.133186E+00    1       1       3.757596E-02    
1       -2.997986077756E+01     -2.234730E+00   1.377127E+01    14      2       3.719130E-01    
2       -3.225469182495E+01     -2.274831E+00   1.014860E+00    19      2       8.177428E-01    
3       -3.227572160076E+01     -2.102978E-02   9.803432E-02    7       1       9.877658E-01    
4       -3.228046895374E+01     -4.747353E-03   8.065159E-03    15      1       1.323322E+00    
5       -3.228102208056E+01     -5.531268E-04   2.835379E-03    12      1       1.596740E+00

...

Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.388333137138E+01     -1.388333E+01   3.634775E+00    1       1       1.109624E-02    
1       -1.560455029101E+01     -1.721219E+00   3.264949E+00    16      2       1.931171E-01    
2       -1.598085969600E+01     -3.763094E-01   1.037497E+00    15      1       3.459392E-01    
3       -1.604736294289E+01     -6.650325E-02   4.886104E-01    15      1       5.109823E-01    
4       -1.606536945698E+01     -1.800651E-02   7.708733E-02    17      1       6.882932E-01    
5       -1.607069162654E+01     -5.322170E-03   4.616661E-02    12      1       8.226230E-01    
6       -1.607271432679E+01     -2.022700E-03   1.067857E-02    14      1       9.690740E-01    
7       -1.607274858685E+01     -3.426006E-05   6.940755E-03    13      1       1.102811E+00    
8       -1.607426838564E+01     -1.519799E-03   3.772282E-04    23      6       1.388730E+00    
9       -1.607427338097E+01   

8       -8.032516604871E+00     -3.632579E-05   1.559831E-05    18      4       4.810762E-01    
9       -8.032516985945E+00     -3.810743E-07   1.441801E-06    11      1       5.322971E-01    
10      -8.032517034430E+00     -4.848490E-08   8.725060E-08    15      1       5.988102E-01    
11      -8.032517039026E+00     -4.595849E-09   1.500921E-08    18      1       6.788752E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.4075977697608538
Chemical potential (eV)  : 11.091300255633755
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -7.752847746105E+00     -7.752848E+00   1.336800E+00    1       1       8.727074E-03    
1       -8.027707101681E+00     -2.748594E-01   2.435923E-01    14      1       7.198882E-02    
2       -8.043037326272E+00     -1.533022E-02   2.009848E-01    12      1       1.263859E-01    
3       -8.067675355150E+00     -2.463803E-02   1.439419E-01    12      2       1.844418E-0

8       -1.604706010529E+01     -8.417587E-08   1.736977E-07    9       1       8.964758E-01    
9       -1.604706011711E+01     -1.182163E-08   1.407687E-08    14      1       1.023591E+00    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.28560740016996755
Chemical potential (eV)  : 7.771773217440917
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -1.458846062175E+01     -1.458846E+01   3.204624E+00    1       1       9.472132E-03    
1       -1.602593777154E+01     -1.437477E+00   9.822557E-01    13      2       1.373110E-01    
2       -1.604564833750E+01     -1.971057E-02   7.384652E-02    7       1       2.035720E-01    
3       -1.604993284170E+01     -4.284504E-03   9.144645E-03    13      1       3.203321E-01    
4       -1.605044949298E+01     -5.166513E-04   2.059347E-03    13      1       4.343450E-01    
5       -1.605061423480E+01     -1.647418E-04   3.251928E-04    16      1       5.752263E-0

4       -4.020911245952E+00     -5.721275E-05   3.253575E-04    16      1       1.499071E-01    
5       -4.020916723386E+00     -5.477434E-06   2.925271E-05    11      1       1.794472E-01    
6       -4.020917479723E+00     -7.563370E-07   1.209756E-05    12      1       2.114069E-01    
7       -4.020918387221E+00     -9.074983E-07   4.556969E-07    19      2       2.625620E-01    
8       -4.020918394405E+00     -7.183543E-09   3.178120E-08    9       1       2.874022E-01    
9       -4.020918395959E+00     -1.554848E-09   2.555441E-09    15      1       3.259883E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.36622377987475097
Chemical potential (eV)  : 9.965456645474744
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -3.801072869396E+00     -3.801073E+00   7.011073E-01    1       1       3.571987E-03    
1       -4.017017578199E+00     -2.159447E-01   1.769318E-01    14      2       4.272604E-0

1       -4.051548997128E+00     -2.087359E-01   1.657369E-01    14      2       4.391074E-02    
2       -4.054427304715E+00     -2.878308E-03   1.571453E-02    9       1       6.919885E-02    
3       -4.054950022017E+00     -5.227173E-04   3.462427E-03    12      1       1.007800E-01    
4       -4.055023248865E+00     -7.322685E-05   3.125112E-04    15      1       1.396689E-01    
5       -4.055030543035E+00     -7.294169E-06   3.622624E-05    12      1       1.718059E-01    
6       -4.055031555534E+00     -1.012499E-06   1.481204E-05    13      1       2.060888E-01    
7       -4.055032715277E+00     -1.159743E-06   1.338859E-06    17      2       2.517879E-01    
8       -4.055032739676E+00     -2.439965E-08   1.003661E-07    8       1       2.741239E-01    
9       -4.055032743922E+00     -4.245671E-09   1.593628E-08    13      1       3.082440E-01    
#### Density Optimization Converged ####
Chemical potential (a.u.): 0.36061184047601597
Chemical potential (eV)  : 9.8127479961

In [41]:
fu_of_ks_KE  = {}
fu_of_ks_TE  = {}
fu_of_ks_DEN = {}
fu_of_ks_AVE = {}
hartree_fukui_KE  = {}
hartree_fukui_TE  = {}
hartree_fukui_DEN = {}
hartree_fukui_AVE = {}

for i, p in enumerate(Phases):
    fu_KE = -fukui_fun(rho_opt_KE[i], rho_opt_charged_KE[i])-1
    fu_TE = -fukui_fun(rho_opt_TE[i], rho_opt_charged_TE[i])-1
    fu_DEN = -fukui_fun(rho_opt_DEN[i], rho_opt_charged_DEN[i])-1
    fu_AVE = -fukui_fun(rho_opt_AVE[i], rho_opt_charged_AVE[i])-1
    
#     fu_KE = - (rho_opt_charged_KE[i]-rho_opt_KE[i])/0.1
    
    fukui_ks = io.read_density(path_file+'/Results/Fukui/'+str(p)+'_fukui.xsf')

    fu_of_ks_KE[p] =(1/2 * (np.abs(fu_KE-fukui_ks)).integral())
    fu_of_ks_TE[p] =(1/2 * (np.abs(fu_TE-fukui_ks)).integral())
    fu_of_ks_DEN[p]=(1/2 * (np.abs(fu_DEN-fukui_ks)).integral())
    fu_of_ks_AVE[p]=(1/2 * (np.abs(fu_AVE-fukui_ks)).integral())
    
    fg_KE   = fu_KE.fft()
    fg_TE   = fu_TE.fft()
    fg_DEN  = fu_DEN.fft()
    fg_AVE  = fu_AVE.fft()

    invg_KE   = fg_KE.grid.gg
    invg_TE   = fg_TE.grid.gg
    invg_DEN  = fg_DEN.grid.gg
    invg_AVE  = fg_AVE.grid.gg
    
    mask_KE   = invg_KE!=0
    mask_TE   = invg_TE!=0
    mask_DEN  = invg_DEN!=0
    mask_AVE  = invg_AVE!=0

    invg_KE[mask_KE ] = 1/invg_KE[mask_KE]
    invg_TE[mask_TE ] = 1/invg_TE[mask_TE]
    invg_DEN[mask_DEN] = 1/invg_DEN[mask_DEN]
    invg_AVE[mask_AVE] = 1/invg_AVE[mask_AVE]

    fu_ha_KE   = ((fg_KE.imag**2+fg_KE.real**2)*4*np.pi*invg_KE).integral()
    fu_ha_TE   = ((fg_TE.imag**2+fg_TE.real**2)*4*np.pi*invg_TE).integral()
    fu_ha_DEN  = ((fg_DEN.imag**2+fg_DEN.real**2)*4*np.pi*invg_DEN).integral()
    fu_ha_AVE  = ((fg_AVE.imag**2+fg_AVE.real**2)*4*np.pi*invg_AVE).integral()

    hartree_fukui_KE[p] = fu_ha_KE
    hartree_fukui_TE[p] = fu_ha_TE
    hartree_fukui_DEN[p] = fu_ha_DEN
    hartree_fukui_AVE[p] = fu_ha_AVE

    print(p, hartree_fukui_KE[p])
    print(hartree(fu_KE).energy)


Si_Btin 0.013436793697291421
0.000564800517415046
Si_fcc 0.011064835962864054
0.0001468008329980029
Si_bcc 0.02252419150375291
0.0004709103463205154
Si_cd 0.0051887599318172165
0.0007734532697152792
Si_dhcp 0.010884875867456206
0.0015598682401548063
Si_hcp 0.02206528865422904
0.00042425554820720853
Si_bct5 0.011983254286939875
0.0008932135908327339
Si_sh 0.053527525049123226
0.00203782941008713


In [42]:
fu_ke = {}
fu_te = {}
fu_den = {}
for i, p in enumerate(Phases):
    fu_ke[p]  = fu_of_ks_KE[p]
    fu_te[p]  = fu_of_ks_TE[p]
    fu_den[p] = fu_of_ks_DEN[p]

In [43]:
from tabulate import tabulate
## Energies wrt KS bcc energies
data = [
    ["Btin", fu_ke['Si_Btin'], fu_te['Si_Btin'], fu_den['Si_Btin']],
    ["bcc",  fu_ke['Si_bcc'], fu_te['Si_bcc'], fu_den['Si_bcc']],
    ["fcc",  fu_ke['Si_fcc'], fu_te['Si_fcc'], fu_den['Si_fcc']],
    ["hcp",  fu_ke['Si_hcp'], fu_te['Si_hcp'], fu_den['Si_hcp']],
    ["bct5", fu_ke['Si_bct5'], fu_te['Si_bct5'], fu_den['Si_bct5']],
    ["sh",   fu_ke['Si_sh'], fu_te['Si_sh'], fu_den['Si_sh']],
    ["cd",   fu_ke['Si_cd'], fu_te['Si_cd'], fu_den['Si_cd']],
    ["dhcp", fu_ke['Si_dhcp'], fu_te['Si_dhcp'], fu_den['Si_dhcp']],
]

table = tabulate(data, headers=[r"Fukui function"," rho0_KIN"," rho0_TE"," rho0_DEN", " N/V", "KS"], tablefmt="grid")

print(table)
# print(tabulate(data, tablefmt="latex"))

+------------------+-------------+------------+-------------+
| Fukui function   |    rho0_KIN |    rho0_TE |    rho0_DEN |
+==================+=============+============+=============+
| Btin             |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| bcc              |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| fcc              |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| hcp              |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| bct5             |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| sh               |           1 |          1 |           1 |
+------------------+-------------+------------+-------------+
| cd               |           1 |          1 |           1 |
+-------

In [30]:
import pickle
with open('/Users/valeria/Documents/aiWT/Final_version/wt/Results/Fukui/Fukui_hartree3.pkl', 'rb') as f:
    ks_fh = pickle.load(f)
    

In [44]:
fu_ha_ke  = {}
fu_ha_te  = {}
fu_ha_den = {}
fu_ha_ave = {}
for i, p in enumerate(Phases):
    fu_ha_ke[p]  = hartree_fukui_KE[p]*100
    fu_ha_te[p]  = hartree_fukui_TE[p]*100
    fu_ha_den[p] = hartree_fukui_DEN[p]*100
    fu_ha_ave[p] = hartree_fukui_AVE[p]*100

In [45]:
from tabulate import tabulate
## Energies wrt KS bcc energies
data = [
    ["Btin", fu_ha_ke['Si_Btin'], fu_ha_te['Si_Btin'], fu_ha_den['Si_Btin'],  fu_ha_ave['Si_Btin'],  ks_fh['Si_Btin']*100 ],
    ["bcc",  fu_ha_ke['Si_bcc'],  fu_ha_te['Si_bcc'],  fu_ha_den['Si_bcc'], fu_ha_ave['Si_bcc'],   ks_fh['Si_bcc']*100 ],
    ["fcc",  fu_ha_ke['Si_fcc'],  fu_ha_te['Si_fcc'],   fu_ha_den['Si_fcc'], fu_ha_ave['Si_fcc'], ks_fh['Si_fcc']*100 ],
    ["hcp",  fu_ha_ke['Si_hcp'],  fu_ha_te['Si_hcp'],  fu_ha_den['Si_hcp'],  fu_ha_ave['Si_hcp'],  ks_fh['Si_hcp']*100 ],
    ["bct5", fu_ha_ke['Si_bct5'], fu_ha_te['Si_bct5'], fu_ha_den['Si_bct5'],   fu_ha_ave['Si_bct5'],   ks_fh['Si_bct5']*100 ],
    ["sh",   fu_ha_ke['Si_sh'],   fu_ha_te['Si_sh'], fu_ha_den['Si_sh'], fu_ha_ave['Si_sh'],  ks_fh['Si_sh']*100 ],
    ["cd",   fu_ha_ke['Si_cd'],   fu_ha_te['Si_cd'],  fu_ha_den['Si_cd'],  fu_ha_ave['Si_cd'], ks_fh['Si_cd']*100 ],
    ["dhcp", fu_ha_ke['Si_dhcp'], fu_ha_te['Si_dhcp'],   fu_ha_den['Si_dhcp'],   fu_ha_ave['Si_dhcp'], ks_fh['Si_dhcp']*100 ],
]

table = tabulate(data, headers=[r"Fukui function"," rho0_KIN"," rho0_TE"," rho0_DEN", " N/V", "KS"], tablefmt="grid")

print(table)
print(tabulate(data, tablefmt="latex"))

+------------------+-------------+------------+-------------+----------+----------+
| Fukui function   |    rho0_KIN |    rho0_TE |    rho0_DEN |      N/V |       KS |
+==================+=============+============+=============+==========+==========+
| Btin             |    1.34368  |   1.33766  |    1.298    | 1.29813  | 1.28694  |
+------------------+-------------+------------+-------------+----------+----------+
| bcc              |    2.25242  |   2.24481  |    2.38372  | 2.41519  | 1.83535  |
+------------------+-------------+------------+-------------+----------+----------+
| fcc              |    1.10648  |   1.10223  |    1.10499  | 1.10305  | 0.990101 |
+------------------+-------------+------------+-------------+----------+----------+
| hcp              |    2.20653  |   2.19819  |    2.19219  | 2.25014  | 2.19007  |
+------------------+-------------+------------+-------------+----------+----------+
| bct5             |    1.19833  |   1.19884  |    1.21228  | 1.14273  | 1.2